In [ ]:
from pulsequantum.livestream.livestream import LiveStream
from pulsequantum.livestream.example_instruments import FilterInstrument


In [ ]:
#Initialize the instruments
filt_instrumment = FilterInstrument("filter")

In [ ]:
voltagecontrolwidget={'phase_x': (filt_instrumment.phase_x,0.1,0),
                      'phase_y': (filt_instrumment.phase_y,0.1,0)}

In [ ]:
#Launch tornade server and refresh it periodicly
#test_org = LiveStream(data_func=filt_instrumment.spectrum,sliders=sliders,port=12344, refresh_period=50)
#test_noise = LiveStream(data_func=filt_instrumment.spectrum_noise,sliders=sliders,port=12345, refresh_period=50)
#test_filter = LiveStream(data_func=filt_instrumment.spectrum_filt,sliders=sliders,voltagecontrolwidget=voltagecontrolwidget, port=12325, refresh_period=500)
test_filter = LiveStream(data_func=filt_instrumment.spectrum_and_noise,voltagecontrolwidget=voltagecontrolwidget, port=12836, refresh_period=500)

In [ ]:
test_filter.set_colobar_scale()

In [ ]:
test_filter.gridspec.grid